<a href="https://colab.research.google.com/github/pavelperc/kaggle_copy/blob/master/HW3_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Данные

In [0]:
!wget https://www.dropbox.com/s/nd7v1fod89xla6j/vk_texts_with_sources.csv

--2019-06-09 12:53:49--  https://www.dropbox.com/s/nd7v1fod89xla6j/vk_texts_with_sources.csv
Resolving www.dropbox.com (www.dropbox.com)... 162.125.81.1, 2620:100:6031:1::a27d:5101
Connecting to www.dropbox.com (www.dropbox.com)|162.125.81.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/nd7v1fod89xla6j/vk_texts_with_sources.csv [following]
--2019-06-09 12:53:50--  https://www.dropbox.com/s/raw/nd7v1fod89xla6j/vk_texts_with_sources.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uccc1ec8de89ad8178e0e4d12b7e.dl.dropboxusercontent.com/cd/0/inline/AifJzAcN8EbgNsOuFwlnUmN4fmPkpNaaBx9sHnnrKn4ewYSngnJopjnaFCJ98e2ykSrgPA0J11hJXtQtTYeF8zVBxco_dBmkH4OJm-R8WlCpeQ/file# [following]
--2019-06-09 12:53:51--  https://uccc1ec8de89ad8178e0e4d12b7e.dl.dropboxusercontent.com/cd/0/inline/AifJzAcN8EbgNsOuFwlnUmN4fmPkpNaaBx9sHnnrKn4ewYSngnJopjnaFCJ98e2ykSrgPA0J11hJXtQtTYeF8zVBxco_d

In [0]:
import pandas as pd
import numpy as np

!pip install pymorphy2[fast]
import pymorphy2

!pip install nltk

import nltk
nltk.download("stopwords")


     |████████████████████████████████| 51kB 2.0MB/s 
     |████████████████████████████████| 7.1MB 8.1MB/s 
     |████████████████████████████████| 256kB 33.4MB/s 
  Stored in directory: /root/.cache/pip/wheels/d4/88/d0/4e4abc83eb8f59a71e8dbd8ba99fd5615a3af1fac1ef7f8825
Successfully built DAWG
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [0]:
df = pd.read_csv('vk_texts_with_sources.csv', usecols = ['text', 'source'])
df = df.dropna()
df.tail()

,text,source
11618,Столичная Госавтоинспекция информирует<br><br>...,mospolice
11620,Сотрудниками ГУУР МВД России перекрыт один из ...,mospolice
11621,Желаем вам хороших выходных в кругу самых близ...,mospolice
11622,Участковый района Коньково задержал молодого ч...,mospolice
11623,На Васильевском спуске проходит митинг-концерт...,mospolice


# Домашнее задание

В этом домашнем задании вы будете решать задачу тематической классификации. Даны тексты, опубликованные в нескольких пабликах VK.com, посвященных государственным и муниципальным службам. Формально задача заключается в том, чтобы по тексту ($d$) определить в каком паблике он опубликован, то есть, к какому классу $c$ он принадлежит. 

## Задание 1 [1 балл]. Описательные статистики
Посчитайте:
* количество текстов и количество классов
* количество слов (без лемматизации и с лемматизацией) в коллекции
* среднюю длину текста в словах и символах
* найдите 5 самых частых существительных в текстах каждого паблика 

*Рекомендуем использовать pandas для расчета описательных статистик.*

In [0]:
print('number of texts:', df.shape[0])
pages = list(df.source.value_counts().index)
print('classes:', pages)

number of texts: 11526
classes: ['mchsgov', 'mil', 'mospolice', 'russianpost']


In [0]:
morph = pymorphy2.MorphAnalyzer()
morph.parse("Сусликов")

[Parse(word='сусликов', tag=OpencorporaTag('NOUN,anim,masc plur,gent'), normal_form='суслик', score=0.3333333333333333, methods_stack=((<DictionaryAnalyzer>, 'сусликов', 2, 7),)),
 Parse(word='сусликов', tag=OpencorporaTag('NOUN,anim,masc plur,accs'), normal_form='суслик', score=0.3333333333333333, methods_stack=((<DictionaryAnalyzer>, 'сусликов', 2, 9),)),
 Parse(word='сусликов', tag=OpencorporaTag('ADJS,Qual masc,sing'), normal_form='сусликовый', score=0.3333333333333333, methods_stack=((<DictionaryAnalyzer>, 'сусликов', 4, 27),))]

In [0]:
from tqdm import tqdm, tqdm_notebook
from nltk.corpus import stopwords
from string import punctuation



words = dict()
normed_words = dict()
nouns_for_pages = dict() # for each type of text

for page in pages:
    nouns_for_pages[page] = []

text_word_counts = []
text_sizes = []
lemmatized_texts = []


morph = pymorphy2.MorphAnalyzer()
russian_stopwords = stopwords.words("russian")


for ind, row in tqdm_notebook(list(df.iterrows())):
#     if ind > 100:
#         break
    
    text = row.text
    page = row.source
    
    lemmatized_words = []
    
    splitted = text.replace('<br>', ' ').split()
    text_sizes.append(len(text))
    text_word_counts.append(len(splitted))
    for word in splitted:
        word = word.strip(punctuation + ' –0123456789«»') # not usual minus, its a dash!
        if word == '':
            continue
#         print(word)
        parsed_word = morph.parse(word)[0]
        normed_word = parsed_word.normal_form
        
        
        if word not in words:
            words[word] = 1
        else:
            words[word] += 1
        
        if normed_word not in normed_words:
            normed_words[normed_word] = 1
        else:
            normed_words[normed_word] += 1
        
        if 'NOUN' in parsed_word.tag:
            nouns_for_pages[page].append(normed_word)
        
        if normed_word not in russian_stopwords:
            lemmatized_words.append(normed_word)
    
    lemmatized_text = ' '.join(lemmatized_words)
    lemmatized_texts.append(lemmatized_text)


    
while len(lemmatized_texts) < df.shape[0]:
    lemmatized_texts.append('')
df['normed'] = lemmatized_texts

# replace the repeated nouns array with the dict of counts
for page in pages:
    grouped = dict()
    arr = nouns_for_pages[page]
    for noun in arr:
        if noun not in grouped:
            grouped[noun] = 1
        else:
            grouped[noun] += 1
    nouns_for_pages[page] = grouped

In [0]:
# df.to_csv('vk_texts_normed.csv')

In [0]:
print('lemmatized:')
df.normed[1].split()[:30]

lemmatized:


['артиллерийский',
 'подразделение',
 'общевойсковой',
 'объединение',
 'юво',
 'ставропольский',
 'край',
 'кубань',
 'зимний',
 'период',
 'обучение',
 'задействовать',
 'тактический',
 'учение',
 'выполнить',
 'огнев',
 'задача',
 'учебный',
 'мероприятие',
 'полигон',
 'николо-александровский',
 'молькино',
 'привлекаться',
 'боевой',
 'расчёт',
 'самоходный',
 'артиллерийский',
 'гаубица',
 'мста-с',
 'акация']

In [0]:
df.head()

,text,source,normed
0,Начальник Главного оперативного управления Генерального штаба Вооруженных Сил РФ генерал-полковник Сергей Рудской провел брифинг для СМИ по ситуации в Сирии <br> <br>С начала этого года сирийскими...,mil,начальник главный оперативный управление генеральный штаб вооружённый сила рф генерал-полковник сергей рудский провести брифинг сми ситуация сирия начало это год сирийский правительственный войско...
1,Артиллерийские подразделения общевойскового объединения ЮВО в Ставропольском крае и на Кубани в зимнем периоде обучения были задействованы в 25 тактических учениях и выполнили более 500 огневых за...,mil,артиллерийский подразделение общевойсковой объединение юво ставропольский край кубань зимний период обучение задействовать тактический учение выполнить огнев задача учебный мероприятие полигон ник...
2,"Подразделения морской пехоты Каспийской флотилии осваивают новые бронетранспортёры БТР-82А, поступившие на вооружение в середине мая<br><br>Механики-водители производят обкатку бронетранспортеров ...",mil,подразделение морской пехота каспийский флотилия осваивать новое бронетранспортёр бтр-82 поступить вооружение середина май механик-водитель производить обкатка бронетранспортёр начало летний перио...
3,"Команды на всеармейских этапах конкурсов АрМИ-2018 «Танковый биатлон» и «Суворовский натиск» в Амурской области не только показывают свои умения управлять военной техникой, но и состязаются в выпо...",mil,команда всеармейский этап конкурс арми танковый биатлон суворовский натиск амурский область показывать свой умение управлять военный техника состязаться выполнение спортивный норматив полигон даль...
4,"На большом учебно-методическом командирском сборе, стартовавшем на нескольких полигонах Западной Сибири, впервые в учебных целях будет оборудован «сирийский вал» — прием, примененный и отработанны...",mil,большой учебно-методический командирский сбор стартовать несколько полигон западный сибирь впервые учебный цель оборудовать сирийский вал — приём применить отработать время военный операция сирия ...


In [0]:
def inv_sort(dict_):
    return sorted(dict_.items(), key=lambda kv: -kv[1])

def print_head(descr, dict_):
    print(descr)
    print('\n'.join(map(str, inv_sort(dict_)[:10])))
    print()


print_head('top words:', words)
print_head('top normed words:', normed_words)

for page in pages:
    print('page:', page)
    print_head('top nouns:', nouns_for_pages[page])

top words:
('в', 31303)
('и', 29198)
('на', 15343)
('по', 13661)
('с', 10719)
('России', 9428)
('В', 7908)
('МВД', 5106)
('из', 4587)
('для', 4008)

top normed words:
('в', 40578)
('и', 29541)
('на', 17195)
('по', 14725)
('с', 12644)
('россия', 9761)
('год', 9410)
('быть', 6259)
('полиция', 6127)
('мвд', 5848)

page: mchsgov
top nouns:
('мчс', 3699)
('мчсроссия', 1879)
('россия', 1720)
('спасатель', 684)
('год', 569)
('человек', 479)
('помощь', 439)
('безопасность', 430)
('работа', 426)
('пожар', 424)

page: mil
top nouns:
('минобороны', 1824)
('военный', 1372)
('россия', 1320)
('год', 1312)
('оборона', 920)
('учение', 890)
('сила', 804)
('корабль', 766)
('флот', 766)
('время', 750)

page: mospolice
top nouns:
('полиция', 5934)
('мвд', 5614)
('год', 5351)
('москва', 4275)
('россия', 4020)
('сотрудник', 3159)
('петровка', 2820)
('увд', 1687)
('служба', 1587)
('дело', 1578)

page: russianpost
top nouns:
('почта', 3268)
('россия', 2701)
('год', 2178)
('отделение', 1630)
('письмо', 1496)
(

In [0]:
print('mean text size:', np.mean(text_sizes) // 1)
print('mean words count:', np.mean(text_word_counts) // 1)

mean text size: 690.0
mean words count: 85.0


Разделите коллекцию текстов на обучающую и тестовую части. 

In [0]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df.drop(columns='source'), df.source.tolist(), random_state=42)

X_train_orig = X_train.text
X_test_orig = X_test.text

X_train = X_train.normed
X_test = X_test.normed

In [0]:
pd.DataFrame(zip(X_test, X_test_orig)).head()

,0,1
0,почта россия подписать соглашение взаимодействие правительство вологодский область май генеральный директор фгуп почта россия дмитрий страшновый посетить вологодский область основный цель визит по...,Почта России подписала соглашение о взаимодействии с Правительством Вологодской области<br> <br>29 мая генеральный директор ФГУП «Почта России» Дмитрий Страшнов посетил Вологодскую область. Основн...
1,экипаж танк маскировать боевой машина тактико-специальный занятие полигон луга,Экипаж танка Т-72 маскирует боевую машину на тактико-специальном занятии на полигоне в Луге
2,юг москва проходить мероприятие направить снижение детский дорожно-транспортный травматизм рамка мероприятие ребёнок дорога сотрудник дпс гибдд увд юао гу мвд россия год москва провести профилакти...,"На юге Москвы проходят мероприятия, направленные на снижение детского дорожно-транспортного травматизма. <br><br>В рамках мероприятия «Ребенок на дороге» сотрудниками ОБ ДПС ГИБДД УВД по ЮАО ГУ МВ..."
3,отправить бесплатно открытка макс выиграть палка селфи сегодня жуковский стартовать международный авиакосмический салон день форум открытый бизнес-общение сми август состояться шоу-программа весь ...,"Отправь бесплатно открытку с МАКСа и выиграй палку для селфи!<br> <br>Сегодня в Жуковском стартует Международный авиакосмический салон. В первые дни форума он открыт для бизнес-общения и СМИ, а 28..."
4,мчс мчсроссия президент рф обсудить главый мчс россия программа дальнейший оснащение министерство http://www.mchs.gov.ru/dop/info/smi/news/item,#МЧС #МЧСРОССИИ <br><br>Президент РФ обсудил с главой МЧС России программу дальнейшего оснащения Министерства.<br><br>http://www.mchs.gov.ru/dop/info/smi/news/item/33476395/


 ## Задание 2 [2 балла]. Классификация по правилам
 
 * Разработайте несколько правил вида "Если встречается слово $w$, то текст относится к паблику $c$"
 * Посчитайте, какую точность, полноту, $f$-меру и $accuracy$ вы получаете при классификации по правилам
 * Получилось ли у вас придумать правило, которое никогда не ошибается?

In [0]:
# returns how much items amoung 'words' are in the text
def check_words(text, words):
    score = 0
    for word in words:
        if word in text:
            score += 1
    return score / len(words)

def max_score(scores):
    return max(scores, key=scores.get)

y_pred = []
for text in X_test:
    scores = dict()
    scores['mchsgov'] = check_words(text, ['мчс', 'мчсроссия', 'спасатель', 'помощь', 'человек', 'безопасность', 'пожар'])
    scores['mil'] = check_words(text, ['военный', 'минобороны', 'оборона', 'учение', 'сила', 'корабль', 'танк'])
    scores['mospolice'] = check_words(text, ['полиция', 'мвд', 'петровка', 'увд', 'дело'])
    scores['russianpost'] = check_words(text, ['почта', 'отделение', 'письмо', 'открытка', 'марка', 'отправление', 'посылка'])
    y_pred.append(max_score(scores))

print('predictions head:')
list(zip(y_pred, y_test))[:10]

predictions head:


[('russianpost', 'russianpost'),
 ('mil', 'mil'),
 ('mospolice', 'mospolice'),
 ('russianpost', 'russianpost'),
 ('mchsgov', 'mchsgov'),
 ('mchsgov', 'mil'),
 ('mchsgov', 'mchsgov'),
 ('mospolice', 'mospolice'),
 ('russianpost', 'russianpost'),
 ('mchsgov', 'mchsgov')]

In [0]:
from sklearn.metrics import precision_score, f1_score, recall_score, accuracy_score, confusion_matrix
import seaborn as sn

labels=['mchsgov', 'mil', 'mospolice', 'russianpost']

def print_conf_matrix(y_test, y_pred):
    conf = confusion_matrix(y_test, y_pred, labels=['mchsgov', 'mil', 'mospolice', 'russianpost'])
    
    print('confusion matrix:')
    print(pd.DataFrame(conf, index=labels, columns=labels))


def print_metrics(y_test, y_pred):
    precision = precision_score(y_test, y_pred, average=None, labels=labels)
    f_score = f1_score(y_test, y_pred, average=None, labels = labels)
    recall = recall_score(y_test, y_pred, average=None, labels = labels)
        
    print(pd.DataFrame([dict(zip(labels, precision)),
                  dict(zip(labels, recall)),
                  dict(zip(labels, f_score))],
                 index=['precision','recall', 'f_score']))
    print('accuracy:', accuracy_score(y_test, y_pred))
    print()
    print_conf_matrix(y_test, y_pred)
    
    
print_metrics(y_test, y_pred)

            mchsgov       mil  mospolice  russianpost
precision  0.766598  0.985149   0.937090     0.970856
recall     0.972368  0.793883   0.982143     0.830218
f_score    0.857309  0.879234   0.959088     0.895046
accuracy: 0.8965995836224844

confusion matrix:
             mchsgov  mil  mospolice  russianpost
mchsgov          739    1          5           15
mil              148  597          7            0
mospolice         12    0        715            1
russianpost       65    8         36          533


## Задание 3 [3 балла]. Baseline
Используйте стандартный ```sklearn.pipeline``` для классификации текстов: 
* векторизация 
* $tf-idf$ взвешивание 
* ваш любимый метод классификации.


Оцените результаты классификации по стандартным мерам качества и проведите анализ ошибок. Для этого рекомендуем визуализировать матрицу ошибок (confusion matrix). 

In [1]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier

pipeline = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', SGDClassifier()),
])

pipeline.fit(X_train, y_train)

y_pred = pipeline.predict(X_test)

print_metrics(y_test, y_pred)

NameError: ignored

## Задание 4 [2 балла]. Снижение размерности
Добавьте в ваш ```sklearn.pipeline```  методы снижения размерности:  PCA / LSI / LSA / LDA / другое. Какие методы классификации разумно использовать после снижения размерности? Как изменились результаты классификации после добавления нового шага?

In [0]:
from sklearn.decomposition import PCA, TruncatedSVD

pipeline = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('svd', TruncatedSVD(n_components=100)),
    ('clf', SGDClassifier()),
])

pipeline.fit(X_train, y_train)

y_pred = pipeline.predict(X_test)

print_metrics(y_test, y_pred)

            mchsgov       mil  mospolice  russianpost
precision  0.978695  0.995918   0.967785     0.946237
recall     0.967105  0.973404   0.990385     0.959502
f_score    0.972866  0.984533   0.978955     0.952823
accuracy: 0.9729354614850798

confusion matrix:
             mchsgov  mil  mospolice  russianpost
mchsgov          735    2          0           23
mil               14  732          0            6
mospolice          0    1        721            6
russianpost        2    0         24          616


## Задание 5 [1 балл]. Лемматизация
Посмотрите, как влияет лемматизация на качество классификации. Как изменится качество классификации, если вы используете ```CountVectorizer``` на словах или $n$-граммах на лемматизированных текстах?

In [0]:
%%time
pipeline = Pipeline([
    ('vect', CountVectorizer(ngram_range=(1, 1))),
    ('tfidf', TfidfTransformer()),
    ('clf', SGDClassifier()),
])

pipeline.fit(X_train_orig, y_train)

y_pred = pipeline.predict(X_test_orig)

print('ORIGINAL TEXT:')

print_metrics(y_test, y_pred)

ORIGINAL TEXT:
            mchsgov       mil  mospolice  russianpost
precision  0.981358  0.994587   0.964048     0.960998
recall     0.969737  0.977394   0.994505     0.959502
f_score    0.975513  0.985915   0.979040     0.960249
accuracy: 0.9757113115891742

confusion matrix:
             mchsgov  mil  mospolice  russianpost
mchsgov          737    2          1           20
mil               12  735          2            3
mospolice          0    2        724            2
russianpost        2    0         24          616
CPU times: user 1.77 s, sys: 210 ms, total: 1.98 s
Wall time: 1.73 s


In [0]:
%%time

pipeline = Pipeline([
    ('vect', CountVectorizer(ngram_range=(1, 1))),
    ('tfidf', TfidfTransformer()),
    ('clf', SGDClassifier()),
])

pipeline.fit(X_train, y_train)

y_pred = pipeline.predict(X_test)

print('LEMMATIZED TEXT:')
print_metrics(y_test, y_pred)

LEMMATIZED TEXT:
            mchsgov       mil  mospolice  russianpost
precision  0.978723  0.998641   0.962815     0.959438
recall     0.968421  0.977394   0.995879     0.957944
f_score    0.973545  0.987903   0.979068     0.958691
accuracy: 0.9753643303261624

confusion matrix:
             mchsgov  mil  mospolice  russianpost
mchsgov          736    1          3           20
mil               13  735          1            3
mospolice          0    0        725            3
russianpost        3    0         24          615
CPU times: user 1.18 s, sys: 154 ms, total: 1.34 s
Wall time: 1.17 s


In [0]:
%%time

pipeline = Pipeline([
    ('vect', CountVectorizer(ngram_range=(1, 2))),
    ('tfidf', TfidfTransformer()),
    ('clf', SGDClassifier()),
])

pipeline.fit(X_train, y_train)

y_pred = pipeline.predict(X_test)

print('LEMMATIZED TEXT WITH NGRAM:')
print_metrics(y_test, y_pred)

LEMMATIZED TEXT WITH NGRAM:
            mchsgov       mil  mospolice  russianpost
precision  0.981383  0.995946   0.966799     0.967033
recall     0.971053  0.980053   1.000000     0.959502
f_score    0.976190  0.987936   0.983120     0.963253
accuracy: 0.9781401804302567

confusion matrix:
             mchsgov  mil  mospolice  russianpost
mchsgov          738    3          1           18
mil               12  737          0            3
mospolice          0    0        728            0
russianpost        2    0         24          616
CPU times: user 4.59 s, sys: 390 ms, total: 4.98 s
Wall time: 4.53 s


In [0]:
pipeline = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', SGDClassifier()),
])

pipeline.fit(X_train, y_train)

y_pred = pipeline.predict(X_test)


pd.set_option('display.max_colwidth', 200)
print('displaying all errors:')
pd.DataFrame(np.transpose([np.array(y_pred)[y_test != y_pred],
                           np.array(y_test)[y_test != y_pred],
                           (X_test)[y_test != y_pred],
                           (X_test_orig)[y_test != y_pred]]), columns=['y_pred', 'y_true', 'normed', 'orig'])

displaying all errors:


,y_pred,y_true,normed,orig
0,mospolice,russianpost,police mvdrussia мвд мвдроссия полиция полицияроссия мвд петровка новость монголия делегация,#police #mvdrussia #МВД #мвдроссии #полиция #полицияроссии #мвд77 #петровка38 #новости #Монголия #делегация
1,mospolice,russianpost,сотрудник полиция южный округа задержать подозревать кража иномарка https://youtu.be/hjj-jzt1ubq police mvdrussia мвд мвдроссия полиция полицияроссия мвд петровка новость юао угон кража,Сотрудники полиции Южного округа задержали подозреваемых в краже иномарки<br><br>https://youtu.be/hjJ-jzT1ubQ<br><br>#police #mvdrussia #МВД #мвдроссии #полиция #полицияроссии #мвд77 #петровка38 #...
2,russianpost,mil,гибель роман филиповый отдать жизнь борьба международный терроризм встрепенул душа миллион человек весь мир однако найтись эйдман который жёлчно усомниться подвиг российский пилот манкурт являться...,"Гибель Романа Филипова, отдавшего жизнь в борьбе с международным терроризмом, встрепенула души миллионов людей во всем мире. Однако нашелся Эйдман, который желчно усомнился в подвиге российского п..."
3,mospolice,russianpost,сотрудник туристический полиция оказать помощь иностранный гражданин сотрудник рота батальон го оперативный полка гу мвд россия год москва время несение служба помочь приезжий женщина который поте...,Сотрудники туристической полиции оказали помощь иностранным гражданам<br><br>Сотрудники 2 роты 3 батальона 1-го оперативного полка ГУ МВД России по г. Москве во время несения службы помогли двум п...
4,mospolice,russianpost,запад столица полицейский задержать трое подозревать грабёж сотрудник уголовный розыск омвд россия район крылатский задержать трое приезжий ближний зарубежье подозрение совершение грабёж омвд росс...,На западе столицы полицейские задержали троих подозреваемых в грабеже<br><br>Сотрудники уголовного розыска ОМВД России по району Крылатское задержали троих приезжих из ближнего зарубежья по подозр...
5,mchsgov,mil,верный друг поздравлять наступить новый год,«Верные друзья» поздравляют вас с наступившим Новым годом!
6,mospolice,russianpost,желать хороший законопослушный выходной фото ый опп гу мвд россия год москва police mvdrussia мвд мвдроссия полиция полицияроссия мвд петровка выходной,Желаем вам хороших и законопослушных выходных!<br><br>Фото: 1-ый ОПП ГУ МВД России по г. Москве<br><br>#police #mvdrussia #МВД #мвдроссии #полиция #полицияроссии #мвд77 #петровка38 #выходные
7,mchsgov,russianpost,комментарий министр связь массовый коммуникация рф николай никифоров повод публикация известие http://izvestia.ru/news,"Комментарий министра связи и массовых коммуникаций РФ Николая Никифорова по поводу публикации в ""Известиях"". http://izvestia.ru/news/571390"
8,mchsgov,mil,мчс мчсроссия сахалин кадетский школа пройти день открытый дверь который учащийся старший класс продемонстрировать свой знание навык область аварийно-спасательный дело http://www.mchs.gov.ru/dop/i...,"#МЧС #МЧСРОССИИ <br><br>На Сахалине в одной их кадетских школ прошел День открытых дверей, на котором учащиеся старших классов продемонстрировали свои знания и навыки в области аварийно-спасательн..."
9,mchsgov,mil,буднимчс мчс мчсроссия большой просто работа фото mary shine,"#БудниМЧС #МЧС #МЧСРОССИИ <br><br>Больше, чем просто работа.<br>Фото: Mary Shine"


## Задание 6 [3 балла]. Continious bag of words
Для baseline решения мы использовали обычное представление текста в виде мешка слов. Попробуйте использовать другие модели представления текста – например, в виде непрерывного мешка слов, то есть, в виде набора эмбеддингов. Для того, чтобы получить вектор текста попробуйте:
* усреднить все эмбеддинги слов, входящих в этот текст
* усреднить все эмбеддинги слов, входящих в этот текст с $tf-idf$ весами
* использовать любую модель эмбеддинга документа.

Используйте любую модель эмбеддингов по вашему вкусу. 


Оцените результаты классификации по стандартным мерам качества и проведите анализ ошибок. Для этого рекомендуем визуализировать матрицу ошибок (confusion matrix). 

In [0]:
cv = CountVectorizer()

vectors = cv.fit_transform(X_train)

vectors

<8644x28491 sparse matrix of type '<class 'numpy.int64'>'
	with 452588 stored elements in Compressed Sparse Row format>

In [0]:
np.mean(vectors, ax)

0.002366741043320613

<8644x28491 sparse matrix of type '<class 'numpy.int64'>'
	with 452588 stored elements in Compressed Sparse Row format>

In [0]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize

import multiprocessing
cores = multiprocessing.cpu_count()

train_tagged = [TaggedDocument(words=text.split(), tags=[i]) for i, text in enumerate(X_train)]
test_tagged = [TaggedDocument(words=text.split(), tags=[i]) for i, text in enumerate(X_test)]

model_dbow = Doc2Vec(dm=0, vector_size=300, negative=5, hs=0, min_count=2, sample = 0, workers=cores)
model_dbow.build_vocab(train_tagged)

In [0]:
test_tagged[0]

TaggedDocument(words=['почта', 'россия', 'подписать', 'соглашение', 'взаимодействие', 'правительство', 'вологодский', 'область', 'май', 'генеральный', 'директор', 'фгуп', 'почта', 'россия', 'дмитрий', 'страшновый', 'посетить', 'вологодский', 'область', 'основный', 'цель', 'визит', 'подписание', 'соглашение', 'правительство', 'вологодский', 'область', 'федеральный', 'государственный', 'унитарный', 'предприятие', 'почта', 'россия', 'взаимодействие', 'сфера', 'совершенствование', 'развитие', 'федеральный', 'почтовый', 'связь', 'территория', 'вологодский', 'область', 'соглашение', 'давать', 'гарантия', 'дальнейший', 'развитие', 'услуга', 'почтовый', 'связь', 'территория', 'вологодский', 'область', 'областной', 'руководство', 'обеспечить', 'координация', 'работа', 'орган', 'местный', 'самоуправление', 'разработка', 'почтовый', 'маршрут', 'размещение', 'почтовый', 'отделение', 'связь', 'соглашение', 'подтверждать', 'правительство', 'область', 'почта', 'россия', 'выходить', 'новый', 'экономич

In [0]:
%%time
model_dbow.train(train_tagged, total_examples=len(train_tagged), epochs=10, )

In [0]:
train_vec = [model_dbow.infer_vector(doc.words, steps=20) for doc in tqdm_notebook(train_tagged)]
test_vec = [model_dbow.infer_vector(doc.words, steps=20) for doc in tqdm_notebook(test_tagged)]


In [0]:
train_vec[0][:10]

array([-0.05165888, -0.07699879,  0.16747276,  0.02872546, -0.0850175 ,
        0.08284292, -0.05571892, -0.18722811,  0.0766766 ,  0.08841929],
      dtype=float32)

In [0]:
cls = SGDClassifier()
cls.fit(train_vec, y_train)
y_pred = cls.predict(test_vec)

print_metrics(y_test, y_pred)

            mchsgov       mil  mospolice  russianpost
precision  0.854061  0.800743   0.879499     0.843895
recall     0.885526  0.860372   0.771978     0.850467
f_score    0.869509  0.829487   0.822238     0.847168
accuracy: 0.842470506592644

confusion matrix:
             mchsgov  mil  mospolice  russianpost
mchsgov          673   48         15           24
mil               42  647         32           31
mospolice         36   84        562           46
russianpost       37   29         30          546


## Задание 7 [2 балла]. fastText

Используйте ```fastText``` в режиме классификации. Оцените результаты классификации по стандартным мерам качества и проведите анализ ошибок. Для этого рекомендуем визуализировать матрицу ошибок (confusion matrix). 

## Задание 8 [4 балла]. CNN

Реализуйте модель Kim et al (2014) для решения задачи классификации с помощью CNN. Оцените результаты классификации по стандартным мерам качества и проведите анализ ошибок. Для этого рекомендуем визуализировать матрицу ошибок (confusion matrix).
Ссылка: Kim Y. Convolutional Neural Networks for Sentence Classification. 2014

## Задание 9 [4 + 2 балла]. RNN

(4 балла)Используйте ```RNN``` (BLSTM с какими-то признаками и пулинг поверх) для решения задачи текстовой классификации. Оцените результаты классификации по стандартным мерам качества и проведите анализ ошибок. Для этого рекомендуем визуализировать матрицу ошибок (confusion matrix).

За дополнительные 2 балла добавьте в модель символьные признаки - CharCNN или CharRNN

## Задание 10 [8 баллов]. ULMFit

Используйте ```ULMFit``` для решения задачи классификации. Оцените результаты классификации по стандартным мерам качества и проведите анализ ошибок. Для этого рекомендуем визуализировать матрицу ошибок (confusion matrix). 

## Конец
Выполните какие-то из предыдущих заданий. Для всех заданий, кроме задания 1 требуется вычислить метрику accuracy метода.

Подведите итоги и проведите сравнение всех использованных методов. Какой из них показался вам лучше и почему?

**NB!** Задание обязательное вне зависимости от того, сколько из предыдущих пунктов вы выполнили, и дополнительных баллов не дает.


Для получения полной оценки за NLP-часть достаточно набрать **20 баллов**.

# Правила сдачи 

1. Домашняя работа должна быть выполнена в ipynb-тетрадке.
2. Сделанную тетрадку нужно отправить ассистенту (ссылка на контакты будет в вики).
3. Задание выполняется индивидуально.
4. Все вычисления должны быть снабжены пояснениями!
5. Дедлайн – 10 июня в 10.00.
